In [ ]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt, seaborn as sns

import IPython.display as ipd

plt.rcParams['figure.figsize'] = (10, 3)

# Data preprocessing

## Features

In [ ]:
def rename_fma_features(features):
    df_features_name = features.iloc[:2,1:]
    new_feature_name = ['track_id']
    for i in range(len(df_features_name.columns)):
        feat = df_features_name.iloc[:,i]
        feat_name = feat.name.split('.')[0]
        stat = feat[0]
        num = feat[1]
        name = feat_name+'_'+num+'_'+stat
        new_feature_name.append(name)
    return_df = features.iloc[3:,:].reset_index(drop=True)
    return_df.columns = new_feature_name
    return return_df

In [ ]:
raw_features = pd.read_csv('/kaggle/input/fma-free-music-archive-small-medium/fma_metadata/features.csv', low_memory=False)
features = rename_fma_features(raw_features)

let's get features with nice and simple feature names

In [ ]:
features = features.apply(pd.to_numeric)

In [ ]:
features

## Labels

- **Genre**

In [ ]:
tracks = pd.read_csv('/kaggle/input/fma-free-music-archive-small-medium/fma_metadata/tracks.csv', index_col=0, low_memory=False)

In [ ]:
# ONLY genre_top, genre as Labels
track_info = tracks[["track.7",'track.8']]
track_info.columns = track_info.iloc[0].rename('track_id')
track_info = track_info.iloc[2:]

In [ ]:
track_info

In [ ]:
track_info.loc[pd.isnull(track_info).any(1)]

In [ ]:
track_info.genre_top.value_counts()

In [ ]:
track_info.genre_top.value_counts().sum()

56976 tracks have no `genre_top`! Only 49598 tracks have specified genre.

But they have specific genre id(in `genres`). 

From `genres.csv`, we can derive the parent genre(=`genre_top`) of each genre id.

In [ ]:
# put parent genre of the first genre in 'genres' to non-specified 'genre_top'

track_info_wo_genre = track_info.loc[pd.isnull(track_info).any(1)]
genres_df = pd.read_csv('/kaggle/input/fma-free-music-archive-small-medium/fma_metadata/genres.csv')
genre_names = []
for i in track_info_wo_genre.genres:
    j = eval(i)+[0]
    if j[0] != 0:
        top_level = genres_df[genres_df.genre_id==j[0]].top_level.values[0]
        parent_genre = genres_df[genres_df.genre_id==top_level]['title'].values[0]
    else:
        parent_genre = np.nan
    genre_names.append(parent_genre)
track_info_ = track_info.copy()
track_info_.loc[track_info_wo_genre.index, 'genre_top'] = genre_names

In [ ]:
track_info_.loc[pd.isnull(track_info_).any(1)]

Now we have 2231 songs that have no genre. Let us just ignore them.

In [ ]:
track_info_.genre_top.value_counts()

In [ ]:
track_info_.genre_top.value_counts().sum()

Put label and features together and drop nan labels

In [ ]:
track_info_.index = pd.to_numeric(track_info_.index)
genres = track_info_[['genre_top']]
data = pd.concat([genres, features.set_index('track_id')], axis=1).dropna()

In [ ]:
data

In [ ]:
# data.to_csv('data_FMA_genre_clf.csv')

# Feature Engineering

## Manual Selection

This procedure is not ideal for data-driven analysis, but based on my experience with MIR, I will select some feature variables manually. This reduces computational cost and makes it easy to analyze feature importance.

In [ ]:
df_ft_mean = data[[col for col in data.columns if 'mean' in col]]

plt.figure(figsize=(10,10))
sns.heatmap(df_ft_mean.corr(), cmap='vlag', vmin=-1, vmax=1,center=0, square=True)
plt.xticks(fontsize = 4)
plt.yticks(fontsize = 4)
plt.show()

- Since chromagram obtained from STFT, CQT, CENS have similar characteristic, I decided to choose only one of them: 'CQT'

- Also for spectral features, I remove bandwidth and rolloff because centroids, bandwidth, and rolloff features are very highly correlated.

In [ ]:
columns_cens = [col for col in data.columns if 'cens' in col]
columns_cstft = [col for col in data.columns if 'stft' in col]
columns_sband = [col for col in data.columns if 'bandwidth' in col]
columns_srolloff = [col for col in data.columns if 'rolloff' in col]

In [ ]:
data_ = data.drop(columns=columns_cens+columns_cstft+columns_sband+columns_srolloff)

In [ ]:
df_ft_mean = data_[[col for col in data_.columns if 'mean' in col]]
plt.figure(figsize=(10,10))
sns.heatmap(df_ft_mean.corr(), cmap='vlag', vmin=-1, vmax=1,center=0, square=True, annot=True, annot_kws={'fontsize':5})
plt.xticks(fontsize = 8)
plt.yticks(fontsize = 8)
plt.show()

## PCA analysis

- with only "mean" features

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, FactorAnalysis

In [ ]:
sc = StandardScaler()
df_ft_mean_scaled = sc.fit_transform(df_ft_mean)

pca = PCA()
ft_pca = pca.fit_transform(df_ft_mean_scaled)

exp_var_pca = pca.explained_variance_ratio_
cum_sum_eigenvalues = np.cumsum(exp_var_pca)

plt.bar(range(0,len(exp_var_pca)), exp_var_pca, alpha=0.5, align='center', label='Individual explained variance')
plt.step(range(0,len(cum_sum_eigenvalues)), cum_sum_eigenvalues, where='mid',label='Cumulative explained variance')
plt.hlines(0.8, xmin=0, xmax=len(cum_sum_eigenvalues), color='red', linewidth=.6)
plt.hlines(0.9, xmin=0, xmax=len(cum_sum_eigenvalues), color='red', linewidth=.6)
plt.ylabel('Explained variance ratio')
plt.xlabel('Principal component index')
plt.legend(loc='best')
plt.tight_layout()
plt.show()

In [ ]:
n_comps = 15 # 0.8 explained variance for pca
feature_names = df_ft_mean.columns

methods = [
    ("PCA", PCA())]

for (method, fa) in methods:
    fig, ax = plt.subplots(figsize=(10,20))
    fa.set_params(n_components=n_comps)
    fa.fit(df_ft_mean_scaled)

    components = fa.components_.T

    vmax = np.abs(components).max()
    sns.heatmap(components, cmap="RdBu_r", vmax=vmax, vmin=-vmax, ax=ax, annot=True)
    ax.set_yticklabels(labels=feature_names, rotation=0, fontdict={'fontsize':8})
    ax.set_xticklabels(labels= range(1, n_comps+1))
    ax.set_title(str(method))
    plt.show()

# Classification with ML models

In [ ]:
from xgboost import XGBClassifier
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, \
ConfusionMatrixDisplay, classification_report

from sklearn.model_selection import train_test_split

## Genre Ambiguity and Label Imbalance

In [ ]:
top_genres = data_.genre_top.value_counts()
top_genres

- The 'International' genre was deleted because it was not related to musical characteristics.

- Genres with less than 1000 songs will be deleted.

In [ ]:
international_ids = data_[data_['genre_top']=='International'].index
data_ = data_.drop(index=international_ids)

In [ ]:
small_genres = top_genres[top_genres<1000].index
for x in small_genres:
    ids = data_[data_['genre_top']==x].index
    data_ = data_.drop(index=ids)

In [ ]:
len(data_)

## Data Split

In [ ]:
raw_X = data_.drop(columns=['genre_top'])

labels = data_.loc[:,'genre_top']
cat_y = pd.Categorical(labels)
y = pd.Series(cat_y.codes)

# train / test
raw_X_train, raw_X_test, y_train, y_test = train_test_split(raw_X, y, test_size=0.3, shuffle=True, random_state=123)

In [ ]:
sc = StandardScaler()
raw_X_scaled_train = sc.fit_transform(raw_X_train)
raw_X_scaled_test = sc.transform(raw_X_test)

pca = PCA()
ft_pca = pca.fit_transform(raw_X_scaled_train)

exp_var_pca = pca.explained_variance_ratio_
cum_sum_eigenvalues = np.cumsum(exp_var_pca)

plt.bar(range(0,len(exp_var_pca)), exp_var_pca, alpha=0.5, align='center', label='Individual explained variance')
plt.step(range(0,len(cum_sum_eigenvalues)), cum_sum_eigenvalues, where='mid',label='Cumulative explained variance')
plt.hlines(0.8, xmin=0, xmax=len(cum_sum_eigenvalues), color='red', linewidth=.6)
plt.hlines(0.9, xmin=0, xmax=len(cum_sum_eigenvalues), color='red', linewidth=.6)
plt.ylabel('Explained variance ratio')
plt.xlabel('Principal component index')
plt.legend(loc='best')
plt.tight_layout()
plt.show()

In [ ]:
n_comps = 60 # 0.8 explained variance for pca
pca = PCA(n_components=n_comps)
X_train = pca.fit_transform(raw_X_scaled_train)
X_train = pd.DataFrame(X_train, index=raw_X_train.index, columns=[f'PCA_{n}' for n in range(1,n_comps+1)])
X_test = pca.transform(raw_X_scaled_test)
X_test = pd.DataFrame(X_test, index=raw_X_test.index, columns=[f'PCA_{n}' for n in range(1,n_comps+1)])

In [ ]:
y_train.value_counts()

In [ ]:
y_test.value_counts()

## Model

### XGBoost

In [ ]:
xgb = XGBClassifier(n_estimators=50)

In [ ]:
xgb.fit(X_train, y_train)

In [ ]:
y_true = y_test.copy()
y_pred = xgb.predict(X_test)

In [ ]:
print('XGB Accuracy: ', accuracy_score(y_true, y_pred))
print('XGB F1: ', f1_score(y_true, y_pred, average='macro'))

In [ ]:
fig, axs = plt.subplots(10, figsize=(10,12), sharex=True)
axs[0].set_title('Confusion Matrix (XGBoost)')
axs[9].set_xlabel('Predicted labels')

for i in range(10):    
    sns.heatmap(confusion_matrix(y_true, y_pred)[i].reshape(1,-1), annot=True, cmap='gray_r',
                xticklabels=cat_y.categories, yticklabels=[cat_y.categories[i]], ax=axs[i])
plt.show()

Most misclassified genre pairs

- Predicted Rock for Blues

- Predicted Electronic for Hip Hop

- Predicted Electronic for Instrumental

- Predicted Experimental for Instrumental

- Predicted Experimental for Jazz

- Predicted Electronic for Pop

- Predicted Rock for Pop

Interpretation

1. The labels consist of mostly Electronic, Experimental and Rock. Therefore the model would predict the most common labels.

2. You can see some similarty in genres like rock and blues, electronic and pop or hip hop.

3. Experimental and Instrumental genres are not really musically dinstiguised genres in my opinion.

In [ ]:
print("XGB classification report:",'\n')
print(classification_report(y_true, y_pred))
print(dict(zip(cat_y.categories, range(10))))

In [ ]:
fig, ax = plt.subplots(figsize=(8,12))
from xgboost import plot_importance
plot_importance(xgb, ax=ax, title='Feature Importance of XGBoost model')
plt.show()

PCA2, PCA5, PCA1 have high scores. As you see the PCA factor loadings (showed in heatmap), MFCC features (mean, max, min, median) seem to be important.

And  spectral features contribute the most to PCA7.